In [1]:
import os, re, requests, logging, calplot
import pandas as pd
import dask.dataframe as dd
import multiprocessing as mp
from dask.multiprocessing import get
from dask.diagnostics import ProgressBar
ProgressBar().register()
from tqdm import tqdm
from itertools import product
from eu_country_codes import COUNTRY_CODES
import ipywidgets as widgets
from IPython.display import HTML
import pandas as pd
import matplotlib.pyplot as plt  # Added import
import ipywidgets as widgets
from IPython.display import display

plt.ioff()

In [2]:
HTML("""\
<style>
.app-subtitle {
    font-size: 1.5em;
}

.app-subtitle a {
    color: #106ba3;
}

.app-subtitle a:hover {
    text-decoration: underline;
}

.app-sidebar p {
    margin-bottom: 1em;
    line-height: 1.7;
}

.app-sidebar a {
    color: #106ba3;
}

.app-sidebar a:hover {
    text-decoration: underline;
}
</style>
""")

In [5]:
import pandas as pd
import matplotlib.pyplot as plt
import calplot
import ipywidgets as widgets
from IPython.display import display

class App:
    def __init__(self, df):
        # Additional data preparation steps
        df['Date'] = pd.to_datetime(df['Date'])
        df['Year'] = df['Date'].dt.year
        df['Month'] = df['Date'].dt.month
        day_order = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
        df['DayOfWeek'] = pd.Categorical(df['Date'].dt.day_name(), categories=day_order, ordered=True)

        self._df = df
        available_cities = self._df['City'].unique()
        available_years = sorted(self._df['Year'].unique(), reverse=True)  # Start from 2023 and go in descending order

        # Dropdowns for filtering
        self._city_dropdown = self._create_indicator_dropdown(available_cities, 0)
        self._year_dropdown = self._create_indicator_dropdown(available_years, 0)

        self._city_dropdown2 = self._create_indicator_dropdown(available_cities, 0)
        self._city_dropdown_compare = self._create_indicator_dropdown(available_cities, 1)
        self._year_dropdown2 = self._create_indicator_dropdown(available_years, 0)

        self._city_dropdown3 = self._create_indicator_dropdown(available_cities, 0)

        self._city_dropdown_calplot = self._create_indicator_dropdown(available_cities, 0)

        self._plot_container = widgets.Output()
        self._plot_container2 = widgets.Output()
        self._plot_container3 = widgets.Output()
        self._plot_container_calplot = widgets.Output()
        self._plot_container_heatmap = widgets.Output()

        self._source_code_button = widgets.Button(description='</> Source Code')
        self._source_code_button.on_click(self._on_source_code_button_click)
        self._source_code_output = widgets.Output(layout={'border': '1px solid black', 'display': 'none'})

        _app_container = widgets.VBox([
            widgets.HTML('<h1 style="font-weight: bold; background-color: #e74944; padding: 10px; color: white;">Max API in Netherlands cities by Year</h1>'),
            widgets.HBox([self._city_dropdown, self._year_dropdown]),
            self._plot_container,
            widgets.HTML('<h1 style="font-weight: bold; background-color: #e74944; padding: 10px; color: white;">Monthly Mean API by City and Year</h1>'),
            widgets.HBox([self._city_dropdown2, self._year_dropdown2, self._city_dropdown_compare]),
            self._plot_container2,
            widgets.HTML('<h1 style="font-weight: bold; background-color: #e74944; padding: 10px; color: white;">Monthly Mean API Levels by Day of the Week</h1>'),
            widgets.HBox([self._city_dropdown3]),
            self._plot_container3,
            widgets.HTML('<h1 style="font-weight: bold; background-color: #e74944; padding: 10px; color: white;">API Time Series</h1>'),
            widgets.HBox([self._city_dropdown_calplot]),
            self._plot_container_calplot,
            widgets.HTML('<h1 style="font-weight: bold; background-color: #e74944; padding: 10px; color: white;">API Time Series Calendar Heatmap</h1>'),
            widgets.HBox([self._city_dropdown_calplot]),
            self._plot_container_heatmap,
            self._source_code_button,
            self._source_code_output
        ], layout=widgets.Layout(align_items='center', flex='3 0 auto'))

        self.container = widgets.VBox([
            widgets.HBox([_app_container])
        ], layout=widgets.Layout(flex='1 1 auto', margin='0 auto 0 auto', max_width='1024px'))

        self._update_app()

    @classmethod
    def from_url(cls, url):
        df = pd.read_csv(url)
        return cls(df)

    def _create_indicator_dropdown(self, indicators, initial_index):
        dropdown = widgets.Dropdown(options=indicators, value=indicators[initial_index])
        dropdown.observe(self._on_change, names=['value'])
        return dropdown

    def _create_plot(self, city, year):
        filtered_data = self._df[(self._df['City'] == city) & (self._df['Year'] == year)]
        max_values = filtered_data.groupby('Month')['API'].idxmax()
        max_value_rows = filtered_data.loc[max_values]
    
        plt.figure(figsize=(10, 6))
        bars = plt.bar(filtered_data['Month'], filtered_data['API'], color='skyblue')
    
        for _, row in max_value_rows.iterrows():
            month = row['Month']
            max_value = row['API']
            plt.text(row['Month'], max_value, f'{max_value}', ha='center', va='bottom')
    
        plt.title(f'API in {city} - {year}')
        plt.xlabel('Month')
        plt.ylabel('API')
    
        # Set y-axis ticks based on the number of unique months
        if len(filtered_data['Month'].unique()) == 1 and year == 2023:
            plt.yticks([12])
    
        plt.show()

    def _create_plot2(self, city, city_compare, year):
        filtered_data = self._df[(self._df['City'] == city) & (self._df['Year'] == year)]
        filtered_data_compare = self._df[(self._df['City'] == city_compare) & (self._df['Year'] == year)]

        monthly_stats = filtered_data.groupby(['Year', 'Month'])['API'].mean().reset_index()
        monthly_stats_compare = filtered_data_compare.groupby(['Year', 'Month'])['API'].mean().reset_index()

        monthly_stats['YearMonth'] = monthly_stats[['Year', 'Month']].apply(lambda x: f'{x[0]}-{x[1]:02}', axis=1)
        monthly_stats_compare['YearMonth'] = monthly_stats_compare[['Year', 'Month']].apply(lambda x: f'{x[0]}-{x[1]:02}', axis=1)

        plt.figure(figsize=(10, 4))
        plt.plot(monthly_stats['YearMonth'], monthly_stats['API'], marker='o', color='skyblue', label=city)
        plt.plot(monthly_stats_compare['YearMonth'], monthly_stats_compare['API'], marker='o', color='orange', label=city_compare)
        plt.title(f'Monthly Mean API in {city} and {city_compare} - {year}')
        plt.xlabel('Year-Month')
        plt.ylabel('Mean API')
        plt.xticks(rotation=45, ha='right')
        plt.legend()

        # Labeling values on the lines
        for x, y in zip(monthly_stats['YearMonth'], monthly_stats['API']):
            plt.text(x, y, f'{y:.2f}', ha='center', va='bottom', color='blue')

        for x, y in zip(monthly_stats_compare['YearMonth'], monthly_stats_compare['API']):
            plt.text(x, y, f'{y:.2f}', ha='center', va='bottom', color='orange')

        plt.tight_layout()
        plt.show()

    def _create_plot3(self, city):
        filtered_data = self._df[(self._df['City'] == city)]
        monthly_stats = filtered_data.groupby(['Month', 'DayOfWeek'])['API'].mean().reset_index()
        pivot_table = monthly_stats.pivot(index='Month', columns='DayOfWeek', values='API')

        plt.figure(figsize=(10, 6))
        plt.imshow(pivot_table, cmap='coolwarm', aspect='auto')
        plt.colorbar(label='Mean API')
        plt.title(f'Monthly Mean API Levels in {city} by Day of the Week')
        plt.xlabel('Day of the Week')
        plt.ylabel('Month')
        plt.xticks(range(len(pivot_table.columns)), pivot_table.columns, rotation=45, ha='right')
        plt.yticks(range(len(pivot_table.index)), pivot_table.index)
        plt.show()

    def _create_plot_calplot(self, city):
        pdTimeSeries = pd.Series(self._df[(self._df['City'] == city) & (self._df['Year'] != 2018)]['API'].values,
                                 index=pd.DatetimeIndex(self._df[(self._df['City'] == city) & (self._df['Year'] != 2018)]['Date']))
        plt.figure(figsize=(12, 6))
        pdTimeSeries.plot(title=f'API Time Series @ {city}', ylabel='API', xlabel='Time', lw=1, grid=True)
        plt.show()

    def _create_plot_heatmap(self, city):
        pdTimeSeries = pd.Series(self._df[(self._df['City'] == city) & (self._df['Year'] != 2018)]['API'].values,
                                 index=pd.DatetimeIndex(self._df[(self._df['City'] == city) & (self._df['Year'] != 2018)]['Date']))
        plt.figure(figsize=(12, 6))
        calplot.calplot(pdTimeSeries, dropzero=True, cmap='coolwarm', suptitle=f'API Time Series Calendar Heatmap @ {city}')
        plt.show()

    def _on_change(self, _):
        city = self._city_dropdown.value
        year = self._year_dropdown.value
        self._plot_container.clear_output(wait=True)
        with self._plot_container:
            self._create_plot(city, year)

        city2 = self._city_dropdown2.value
        city_compare = self._city_dropdown_compare.value
        year2 = self._year_dropdown2.value
        self._plot_container2.clear_output(wait=True)
        with self._plot_container2:
            self._create_plot2(city2, city_compare, year2)

        city3 = self._city_dropdown3.value
        self._plot_container3.clear_output(wait=True)
        with self._plot_container3:
            self._create_plot3(city3)

        city_calplot = self._city_dropdown_calplot.value
        self._plot_container_calplot.clear_output(wait=True)
        with self._plot_container_calplot:
            self._create_plot_calplot(city_calplot)

        city_heatmap = self._city_dropdown_calplot.value
        self._plot_container_heatmap.clear_output(wait=True)
        with self._plot_container_heatmap:
            self._create_plot_heatmap(city_heatmap)

    def _update_app(self):
        city = self._city_dropdown.value
        year = self._year_dropdown.value
        self._plot_container.clear_output(wait=True)
        with self._plot_container:
            self._create_plot(city, year)

        city2 = self._city_dropdown2.value
        city_compare = self._city_dropdown_compare.value
        year2 = self._year_dropdown2.value
        self._plot_container2.clear_output(wait=True)
        with self._plot_container2:
            self._create_plot2(city2, city_compare, year2)

        city3 = self._city_dropdown3.value
        self._plot_container3.clear_output(wait=True)
        with self._plot_container3:
            self._create_plot3(city3)

        city_calplot = self._city_dropdown_calplot.value
        self._plot_container_calplot.clear_output(wait=True)
        with self._plot_container_calplot:
            self._create_plot_calplot(city_calplot)

        city_heatmap = self._city_dropdown_calplot.value
        self._plot_container_heatmap.clear_output(wait=True)
        with self._plot_container_heatmap:
            self._create_plot_heatmap(city_heatmap)

    def _generate_source_code(self):
        source_code = """
# Load required Python packages
import os, re, requests, logging, calplot
import pandas as pd
import dask.dataframe as dd
import multiprocessing as mp
from dask.multiprocessing import get
from dask.diagnostics import ProgressBar
ProgressBar().register()
from tqdm import tqdm
from itertools import product
from eu_country_codes import COUNTRY_CODES

# Function to download remote file to the disk
def urlDownload(urlLink, showProgress = False):
  with requests.get(urlLink, stream=True) as r:
    fileSize = int(r.headers.get('Content-Length'))
    fileName = r.headers.get('Content-Disposition').split("filename=")[1]
    if not os.path.exists(fileName) or os.path.getsize(fileName) != fileSize:
      block_size = 1024
      if showProgress:
        print(f"Downloading {fileName}")
        progress_bar = tqdm(total=fileSize, unit='iB', unit_scale=True)
      with open(fileName, 'wb') as file:
        for data in r.iter_content(block_size):
          if showProgress:
            progress_bar.update(len(data))
          file.write(data)
      if showProgress:
        progress_bar.close()
    return fileName
    
# import requests
# from tqdm import tqdm  # Import tqdm for the progress bar

def urlDownload(urlLink, showProgress=False):
    with requests.get(urlLink, stream=True) as r:
        fileSize = int(r.headers.get('Content-Length'))
        fileName = r.headers.get('Content-Disposition').split("filename=")[1]

        with open(fileName, 'wb') as f, tqdm(total=fileSize, unit='B', unit_scale=True, unit_divisor=1024, disable=not showProgress) as bar:
            for data in r.iter_content(chunk_size=1024):
                bar.update(len(data))
                f.write(data)

    return fileName

# Download the newest data
urlLocation = 'https://aqicn.org/data-platform/covid19/report/39374-7694ec07/'
csvFile = urlDownload(urlLocation, showProgress=True)
print(csvFile)
# import pandas as pd
# from itertools import product  # Import the product function

# Create lists of year and quarter names
yNames = [str(i) for i in range(2019, 2022)]
qNames = ["Q" + str(i) for i in range(1, 5)]

# Create a data frame with the url locations and year/quarter combinations
DF = pd.DataFrame(list(product(yNames, qNames)), columns=['yNames', 'qNames'])
DF.insert(loc=0, column='urlLocation', value=urlLocation)

# Combine url location and year/quarter combinations into a single column
DF['urlLocations'] = DF['urlLocation'] + DF['yNames'] + DF['qNames']
DF.drop(['urlLocation', 'yNames', 'qNames'], axis=1, inplace=True)

DF

# Download legacy data (sequentially)
csvFiles = DF.apply(lambda x : urlDownload(x[0], showProgress = True), axis=1)

# import os  # Import the os module
# Delete downloaded legacy data files
for delFile in csvFiles:
    os.unlink(delFile)
    
# Download legacy data (in parallel)
import dask.dataframe as dd
import multiprocessing as mp  # Import multiprocessing module
DDF = dd.from_pandas(DF, npartitions=mp.cpu_count())
csvFiles = DDF.apply(lambda x : urlDownload(x[0]), axis=1, meta=pd.Series(dtype="str")).compute(scheduler='threads')

# Define the columns to load
meta_cols = ['Date', 'Country', 'City', 'Specie']
main_column = 'median' # 'count', 'min', 'max', 'median', 'variance'
selected_cols = meta_cols + [main_column]

# Read the newest data file and skip the first 4 lines
DF = pd.read_csv(csvFile, skiprows=4, usecols=selected_cols)

# Leave EU data, rename main column to Value
selectEU = DF['Country'].isin(COUNTRY_CODES)
newTable = DF[selectEU].rename(columns={main_column: 'Value'})
print(newTable)

# Read legacy data files (sequentially)
fileNamesQ = [f for f in os.listdir('.') if re.match(r'^.*Q\d.csv$', f)]
DF = pd.concat((pd.read_csv(f, skiprows=4, usecols=selected_cols) for f in fileNamesQ), ignore_index=True)
selectEU = DF['Country'].isin(COUNTRY_CODES)
oldTable = DF[selectEU].rename(columns={main_column: 'Value'})
print(oldTable)

# Read legacy data files (in parallel)
fileNamesQ = [f for f in os.listdir('.') if re.match(r'^.*Q\d.csv$', f)]
DF = dd.compute(dd.read_csv(fileNamesQ, skiprows=4, usecols=selected_cols))[0]
selectEU = DF['Country'].isin(COUNTRY_CODES)
oldTable = DF[selectEU].rename(columns={main_column: 'Value'})
print(oldTable)

specie_summary = oldTable['Specie'].value_counts()

# Print the summary
print(specie_summary)

DF = pd.concat([oldTable, newTable])

# Sorting and removing duplicates
dataTableEU = DF.sort_values(by=['Country', 'City', 'Date']).drop_duplicates()

# Filtering for Netherlands
dataTableNetherlands = dataTableEU[dataTableEU['Country'] == 'NL']

# Keeping only specified columns
selected_columns = ['pm10', 'pm25', 'no2', 'o3', 'so2', 'co']
dataTableNetherlands = dataTableNetherlands[dataTableNetherlands['Specie'].isin(selected_columns)]

# Print the DataFrame for Netherlands with selected columns
print(dataTableNetherlands)

# Descriptive statistics for daily values of selected variables
selectedVars = ['pm10', 'pm25', 'no2', 'o3', 'so2', 'co']
selectedIdx = dataTableNetherlands['Specie'].isin(selectedVars)
dataTableNetherlands = dataTableNetherlands[selectedIdx]
dataTableNetherlands.groupby('Specie')['Value'].describe()

# 2021-10-03 Barcelona fix
print(dataTableNetherlands)
dataTableNetherlands = dataTableNetherlands.groupby(['Date', 'Country', 'City', 'Specie'])[['Value']].mean().reset_index()
print(dataTableNetherlands)

dataTableNetherlandsAPI = dataTableNetherlands.pivot_table(index=['Date', 'Country', 'City'], columns='Specie', values='Value').reset_index()
dataTableNetherlandsAPI["API"] = dataTableNetherlandsAPI[['pm10', 'pm25', 'no2', 'o3', 'so2', 'co']].max(axis=1)
dataTableNetherlandsAPI = dataTableNetherlandsAPI.dropna(subset=["API"])
print(dataTableNetherlandsAPI)
dataTableNetherlandsAPI.to_csv(r"/Users/myPath/Name1.csv", index=False)

import pandas as pd

dataTableNetherlandsAPI['Date'] = pd.to_datetime(dataTableNetherlandsAPI['Date'])
dataTableNetherlandsAPI['Year'] = dataTableNetherlandsAPI['Date'].dt.year
dataTableNetherlandsAPI['Month'] = dataTableNetherlandsAPI['Date'].dt.month
top_5_highest_api = dataTableNetherlandsAPI.nlargest(5, 'API')
print("Top 5 results with the highest API in the Netherlands:")
print(top_5_highest_api[['Year', 'Month', 'API']])

# import matplotlib.pyplot as plt
max_api_row = pd.DataFrame(dataTableNetherlandsAPI)

selected_year = max_api_row['Year'].iloc[0]
selected_month = max_api_row['Month'].iloc[0]

selected_data = dataTableNetherlandsAPI[(dataTableNetherlandsAPI['Year'] == selected_year) & (dataTableNetherlandsAPI['Month'] == selected_month)]
sorted_data = selected_data.sort_values(by='API', ascending=False)

plt.figure(figsize=(10, 6))
plt.bar(sorted_data['City'], sorted_data['API'], color='skyblue')
plt.title(f'Highest API Cities in {selected_year}-{selected_month}')
plt.xlabel('City')
plt.ylabel('API')
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()

# Descriptive statistics for yearly API
myCity = 'Maastricht'
myTable = dataTableNetherlandsAPI[(dataTableNetherlandsAPI['City']==myCity) & (dataTableNetherlandsAPI['Date']>='2019-01-01')][["Date", "API"]]
myTable.groupby(pd.to_datetime(myTable['Date']).dt.year).describe()

myTable['Year'] = pd.to_datetime(myTable['Date']).dt.year
myTable['Month'] = pd.to_datetime(myTable['Date']).dt.month

monthly_stats = myTable.groupby(['Year', 'Month'])['API'].mean().reset_index()

monthly_stats['YearMonth'] = monthly_stats[['Year', 'Month']].apply(lambda x: f'{x[0]}-{x[1]:02}', axis=1)

plt.figure(figsize=(12, 6))
plt.plot(monthly_stats['YearMonth'], monthly_stats['API'], marker='o', color='skyblue', label=myCity)
plt.title(f'Monthly Mean API in {myCity}')
plt.xlabel('Year-Month')
plt.ylabel('Mean API')
plt.xticks(rotation=45, ha='right')
plt.legend()
plt.tight_layout()
plt.show()

myTable['Date'] = pd.to_datetime(myTable['Date'])
myTable['Month'] = myTable['Date'].dt.month

day_order = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
myTable['DayOfWeek'] = pd.Categorical(myTable['Date'].dt.day_name(), categories=day_order, ordered=True)

monthly_stats = myTable.groupby(['Month', 'DayOfWeek'])['API'].mean().reset_index()

pivot_table = monthly_stats.pivot(index='Month', columns='DayOfWeek', values='API')

plt.figure(figsize=(10, 6))
plt.imshow(pivot_table, cmap='coolwarm', aspect='auto')
plt.colorbar(label='Mean API')
plt.title('Monthly Mean API Levels in Maastricht by Day of the Week')
plt.xlabel('Day of the Week')
plt.ylabel('Month')
plt.xticks(range(len(pivot_table.columns)), pivot_table.columns, rotation=45, ha='right')
plt.yticks(range(len(pivot_table.index)), pivot_table.index)
plt.show()

# Plot API time series calendar heatmap
pdTimeSeries = pd.Series(myTable['API'].values, index=pd.DatetimeIndex(myTable['Date']))
logging.getLogger('matplotlib.font_manager').disabled = True
calplot.calplot(pdTimeSeries, dropzero=True, cmap='coolwarm', suptitle='API @ ' + myCity)

# Plot API time series dynamics
pdTimeSeries.plot(title='API @ ' + myCity, ylabel="API", xlabel="Time", lw=1, grid=True)
        """
        return source_code

    def _on_source_code_button_click(self, _):
        with self._source_code_output:
            self._source_code_output.clear_output(wait=True)
            if self._source_code_output.layout.display == 'none':
                source_code = self._generate_source_code()
                print(source_code)
                self._source_code_output.layout.display = ''
            else:
                self._source_code_output.layout.display = 'none'

    def _close_textbox(self, _):
        with self._source_code_output:
            self._source_code_output.clear_output(wait=True)
            self._source_code_output.layout.display = 'none'

# Load your data
file_path = '/Users/myPath/Name1.csv'
dataTableNetherlandsAPI = pd.read_csv(file_path)

# Create an instance of the App
app_instance = App(dataTableNetherlandsAPI)

# Display the app container
display(app_instance.container)